In [1]:
import pandas as pd
import numpy as np
import bq_helper
from bq_helper import BigQueryHelper

In [3]:
noaa_gsod = bq_helper.BigQueryHelper(active_project= "bigquery-public-data", 
                                     dataset_name= "noaa_gsod")


In [4]:
bq_assistant = BigQueryHelper("bigquery-public-data", "noaa_gsod")

bq_assistant.list_tables()

['gsod1929',
 'gsod1930',
 'gsod1931',
 'gsod1932',
 'gsod1933',
 'gsod1934',
 'gsod1935',
 'gsod1936',
 'gsod1937',
 'gsod1938',
 'gsod1939',
 'gsod1940',
 'gsod1941',
 'gsod1942',
 'gsod1943',
 'gsod1944',
 'gsod1945',
 'gsod1946',
 'gsod1947',
 'gsod1948',
 'gsod1949',
 'gsod1950',
 'gsod1951',
 'gsod1952',
 'gsod1953',
 'gsod1954',
 'gsod1955',
 'gsod1956',
 'gsod1957',
 'gsod1958',
 'gsod1959',
 'gsod1960',
 'gsod1961',
 'gsod1962',
 'gsod1963',
 'gsod1964',
 'gsod1965',
 'gsod1966',
 'gsod1967',
 'gsod1968',
 'gsod1969',
 'gsod1970',
 'gsod1971',
 'gsod1972',
 'gsod1973',
 'gsod1974',
 'gsod1975',
 'gsod1976',
 'gsod1977',
 'gsod1978',
 'gsod1979',
 'gsod1980',
 'gsod1981',
 'gsod1982',
 'gsod1983',
 'gsod1984',
 'gsod1985',
 'gsod1986',
 'gsod1987',
 'gsod1988',
 'gsod1989',
 'gsod1990',
 'gsod1991',
 'gsod1992',
 'gsod1993',
 'gsod1994',
 'gsod1995',
 'gsod1996',
 'gsod1997',
 'gsod1998',
 'gsod1999',
 'gsod2000',
 'gsod2001',
 'gsod2002',
 'gsod2003',
 'gsod2004',
 'gsod2005',

In [37]:
query1 = """
            SELECT 
                usaf AS Station_numbers, 
                lat AS Latitude, 
                lon AS Longitude, 
            FROM 
                `bigquery-public-data.noaa_gsod.stations` 
            WHERE 
                country = 'US' AND lat IS NOT NULL AND lon IS NOT NULL AND NOT (lat = 0.0 AND lon = 0.0) 
            ORDER BY 
                usaf
        """
query2 = """
            SELECT 
                stn AS Station_number, 
                year AS Year, 
                mo AS Month, 
                da AS day,
                temp AS Mean_temp,
            FROM 
                `bigquery-public-data.noaa_gsod.gsod2020`  
        """



In [29]:
stations = noaa_gsod.query_to_pandas_safe(query1, max_gb_scanned=10)

In [38]:
gsod2020 = noaa_gsod.query_to_pandas_safe(query2, max_gb_scanned=10)

In [46]:
gsod2020 = gsod2020.loc[gsod2020.Station_number ==stations.Station_numbers]

ValueError: Can only compare identically-labeled Series objects

In [42]:
gsod2020.head()

,Station_number,Year,Month,day,Mean_temp
0,170700,2020,05,01,58.3
1,406220,2020,05,01,71.5
2,420830,2020,02,01,37.4
3,423990,2020,05,01,83.4
4,428910,2020,05,01,82.1


In [43]:
stations

,Station_numbers,Latitude,Longitude
0,621010,50.600,-2.933
1,621110,58.900,-0.200
2,621130,58.400,0.300
3,621160,58.100,1.800
4,621170,57.900,0.100
5,621220,56.500,2.017
6,621250,53.800,-3.800
7,621260,53.800,-3.600
8,621300,53.000,1.700
9,621370,57.350,1.467
